---
title: "FOIA Document OCR Processing"
subtitle: "Processing Murphy & Tong FOIA Documents about State DNA Database Racial Composition"
author: "Tina Lasisi"
date: today
format:
  html:
    toc: true
    toc-depth: 4
    toc-expand: 3
    toc-location: left   # margin TOC → uses website label
    number-sections: true
language:
  toc-title-website: ""  # <- blank kills the “On this page” header
execute:
  echo: true
  warning: false
---


# Overview

This document details the processing of Freedom of Information Act (FOIA) responses from seven U.S. states regarding the demographic composition of their State DNA Index System (SDIS) databases. These responses were obtained by Professor Erin Murphy (NYU Law) in 2018 as part of research on racial disparities in DNA databases.

# Materials and Methods

## Data Sources

### Raw FOIA Responses

The original FOIA responses are stored in two formats:

- **PDFs**: `raw/foia_pdfs/` - Original scanned documents
- **HTML**: `raw/foia_html/` - OCR'd versions for easier extraction

States included:

- California
- Florida  
- Indiana
- Maine
- Nevada
- South Dakota
- Texas

## File Structure and Contents

### State-Specific Files: `per_state/[state]_foia_data.csv`

**Purpose**: Individual files for each state containing only their reported data.

**Structure**: Long format with columns:

- `state`: State name
- `offender_type`: Category of individuals (Convicted Offender, Arrestee, Combined, etc.)
- `variable_category`: Type of data (total, gender, race, gender_race)
- `variable_detailed`: Specific value (e.g., Male, Female, African American)
- `value`: The reported number or percentage
- `value_type`: Whether value is a "count" or "percentage"
- `date`: Date of data snapshot, if reported

### State Processing


In [ ]:
#| label: setup
#| echo: true

import pandas as pd
from pathlib import Path
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import numpy as np

# Display options
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# Path to per‑state files (run notebook from analysis/)
base_dir   = Path("..")
per_state  = base_dir / "output" / "foia" / "per_state"

# ------------------------------------------------------------------
# 1. Discover available per‑state CSV files
# ------------------------------------------------------------------
state_files = sorted(per_state.glob("*_foia_data.csv"))

if not state_files:
    raise FileNotFoundError(
        f"No per‑state FOIA files found in {per_state}. "
        "Check the folder path."
    )

def stem_to_state(stem: str) -> str:
    toks = stem.split("_")
    if "foia" in toks:
        toks = toks[:toks.index("foia")]
    return " ".join(t.title() for t in toks)

states_available = [stem_to_state(f.stem) for f in state_files]


print(f"✓ Found {len(state_files)} per‑state files:")
for s in states_available:
    print("  •", s)

display(Markdown(
    f"**States with data loaded:** {', '.join(states_available)}"
))

# ------------------------------------------------------------------
# 2. Initialise empty containers for the loop that follows
# ------------------------------------------------------------------
foia_combined       = pd.DataFrame()   # merged tidy data
foia_state_metadata = []               # list of dicts, one per state

## Processing workflow

For transparency, each state file is processed independently then merged into a single **combined long‑format table (`foia_combined`)**:

1. **Load one file per state** from `output/foia/per_state/`.
2. **Append** its rows to `foia_combined`.
   A parallel dataframe, **`foia_state_metadata`**, records what each state reported (counts, percentages, which categories) and any state-specific characteristics (e.g. Nevada's "flags" terminology).
3. **Quality‑check each state**:

   * verify that race and gender percentages sum to ≈ 100 % when provided,
   * confirm that demographic counts sum to the state's reported total profiles,
   * **calculate** any missing counts or percentages and tag those rows `value_source = "calculated"`.
4. **Save outputs**

   * `output/foia/foia_data_clean.csv` — the fully combined tidy table with both reported and calculated values,
   * `output/foia/foia_state_metadata.csv` — one row per state summarising coverage and caveats.
     After QC passes, I will freeze `foia_data_clean.csv` to `data/v1.0/foia_state_race_v1.0.csv`.


## Helper Functions

The functions below perform each transformation required for harmonizing the state‑level FOIA tables.  
Every step is documented for transparency and reproducibility.


In [ ]:
#| label: helpers
#| echo: true

import pandas as pd
from IPython.display import Markdown, display

# Columns retained from every raw table
COLS_NEEDED = [
    "state", "offender_type", "variable_category",
    "variable_detailed", "value", "value_type"
]

# ------------------------------------------------------------------
# 1. Ingest and numeric coercion
# ------------------------------------------------------------------
def load_state(path):
    """
    Read a *_foia_data.csv* file, enforce column order,
    and convert <1 to 0.5 so that trace counts are retained.
    Execution halts if non‑numeric values remain.
    """
    df = pd.read_csv(path)
    if "state" not in df.columns:
        df["state"] = (
            path.stem.replace("_foia_data", "")
            .replace("_", " ").title()
        )
    df = df[COLS_NEEDED]
    df["value_source"] = "reported"

    df["value"] = (
        df["value"].replace({"<1": 0.5})
        .apply(pd.to_numeric, errors="coerce")
    )
    nonnumeric = df[df["value"].isna()]
    if not nonnumeric.empty:
        display(Markdown(f"**Non‑numeric rows in {path.name}; please amend**"))
        display(nonnumeric)
        raise ValueError("Numeric coercion failure")
    return df

# ------------------------------------------------------------------
# 2. Fill missing Male counts and Unknown race counts
# ------------------------------------------------------------------
def fill_demographic_gaps(df):
    """
    If exactly one gender or the Unknown race category is absent and
    totals permit a residual, calculate and insert the missing count.
    """
    inserts = []
    for ot in df.offender_type.unique():
        tot = df.query(
            "offender_type == @ot and variable_category == 'total' and "
            "variable_detailed == 'total_profiles' and value_type == 'count'"
        )
        if tot.empty:
            continue
        total = tot["value"].iat[0]

        # gender residual ----------------------------------------------
        g = df.query(
            "offender_type == @ot and variable_category == 'gender' and "
            "value_type == 'count'"
        )
        missing_gender = {"Male", "Female"} - set(g.variable_detailed)
        if len(g) == 1 and missing_gender:
            inserts.append({
                "state"            : df.state.iat[0],
                "offender_type"    : ot,
                "variable_category": "gender",
                "variable_detailed": missing_gender.pop(),
                "value"            : total - g.value.sum(),
                "value_type"       : "count",
                "value_source"     : "calculated"
            })

        # race residual -----------------------------------------------
        r = df.query(
            "offender_type == @ot and variable_category == 'race' and "
            "value_type == 'count'"
        )
        if not r.empty and "Unknown" not in r.variable_detailed.values:
            gap = total - r.value.sum()
            if gap > 0:
                inserts.append({
                    "state"            : df.state.iat[0],
                    "offender_type"    : ot,
                    "variable_category": "race",
                    "variable_detailed": "Unknown",
                    "value"            : gap,
                    "value_type"       : "count",
                    "value_source"     : "calculated"
                })
    if inserts:
        df = pd.concat([df, pd.DataFrame(inserts)], ignore_index=True)
    return df

# ------------------------------------------------------------------
# 3. Construct Combined offender type if absent
# ------------------------------------------------------------------
def add_combined(df):
    """
    When a state reports Convicted Offender and Arrestee counts but
    omits Combined, create a Combined block by summing the two.
    """
    if "Combined" in df.offender_type.unique():
        return df
    required = {"Convicted Offender", "Arrestee"}
    if not required.issubset(df.offender_type.unique()):
        return df  # cannot construct

    grp_cols = ["variable_category", "variable_detailed", "value_type"]
    summed = (
        df.query("value_type == 'count'")
        .groupby(grp_cols, as_index=False)["value"].sum()
    )
    summed["state"]         = df.state.iat[0]
    summed["offender_type"] = "Combined"
    summed["value_source"]  = "calculated"
    return pd.concat([df, summed], ignore_index=True)

# ------------------------------------------------------------------
# 4. Derive percentages wherever only counts exist
# ------------------------------------------------------------------
def add_percentages(df):
    """
    Ensure that every gender and race row has both count and percentage
    values, derived from the offender‑type total if necessary.
    """
    totals = (
        df.query(
            "variable_category == 'total' and "
            "variable_detailed == 'total_profiles' and "
            "value_type == 'count'"
        )
        .set_index("offender_type")["value"]
        .to_dict()
    )
    new_pct_rows = []
    need_pct = df.query(
        "value_type == 'count' and variable_category != 'total'"
    )
    for _, row in need_pct.iterrows():
        exists = df[
            (df.state == row.state) &
            (df.offender_type == row.offender_type) &
            (df.variable_category == row.variable_category) &
            (df.variable_detailed == row.variable_detailed) &
            (df.value_type == "percentage")
        ]
        if not exists.empty:
            continue
        pct_value = round(
            row.value / totals[row.offender_type] * 100, 2
        )
        new_pct_rows.append({
            **row,
            "value"       : pct_value,
            "value_type"  : "percentage",
            "value_source": "calculated"
        })
    if new_pct_rows:
        df = pd.concat([df, pd.DataFrame(new_pct_rows)], ignore_index=True)
    return df

# ------------------------------------------------------------------
# 5. Quality‑control utilities
# ------------------------------------------------------------------
def counts_consistent(df):
    """
    Verifies that demographic counts sum to total_profiles for each
    offender type and category.
    """
    demo_sum = (
        df.query("value_type == 'count' and variable_category != 'total'")
        .groupby(["offender_type", "variable_category"])["value"]
        .sum()
    )
    totals = (
        df.query(
            "variable_category == 'total' and "
            "variable_detailed == 'total_profiles' and "
            "value_type == 'count'"
        )
        .set_index("offender_type")["value"]
    )
    return all(
        abs(demo_sum.loc[idx] - totals[idx[0]]) < 1e-6
        for idx in demo_sum.index
    )

def percentages_consistent(df):
    """
    Verifies that derived or reported percentages sum to 100 ± 0.5 %.
    """
    for (ot, cat), grp in df.query(
        "value_type == 'percentage'"
    ).groupby(["offender_type", "variable_category"]):
        if abs(grp.value.sum() - 100) > 0.5:
            return False
    return True





----

## Helper Functions

```{python}
#| label: helper-functions
#| echo: true

# Define columns needed for foia_combined
COLUMNS_NEEDED = ['state', 'offender_type', 'variable_category', 
                  'variable_detailed', 'value', 'value_type']

def report_status(df, category):
    values = df.loc[df["variable_category"] == category, "value_type"].unique()
    if {"count", "percentage"}.issubset(values):
        return "both"
    elif "count" in values:
        return "counts"
    elif "percentage" in values:
        return "percentages"
    else:
        return "neither"

def verify_category_totals(df):
    """
    For every offender_type × variable_category pair:
      • find the reported total_profiles,
      • sum all count rows in that category,
      • return the difference.
    """
    # 1  pull total_profiles per offender_type
    total_map = (df[(df["variable_category"] == "total") &
                    (df["variable_detailed"] == "total_profiles")]
                 .set_index("offender_type")["value"]
                 .to_dict())

    # 2  sum counts by offender_type and variable_category
    demo_sum = (df[(df["value_type"] == "count") &
                   (df["variable_category"] != "total")]
                .groupby(["offender_type", "variable_category"])["value"]
                .sum()
                .rename("sum_demo_counts")
                .reset_index())

    # 3  attach total_profiles and compute difference
    demo_sum["total_profiles"] = demo_sum["offender_type"].map(total_map)
    demo_sum["difference"] = demo_sum["total_profiles"] - demo_sum["sum_demo_counts"]

    # tidy columns order
    return demo_sum[["offender_type",
                     "variable_category",
                     "total_profiles",
                     "sum_demo_counts",
                     "difference"]]

def calculate_combined_totals(df, state_name):
    """
    Calculate Combined totals by summing across offender types.
    Returns a dataframe of Combined rows to add.
    """
    # Get all counts
    counts_df = df[(df['value_type'] == 'count')].copy()
    
    # Group by variable_category and variable_detailed, sum values
    combined_sums = (counts_df.groupby(['variable_category', 'variable_detailed'])['value']
                     .sum()
                     .reset_index())
    
    # Create Combined rows
    combined_rows = []
    for _, row in combined_sums.iterrows():
        combined_rows.append({
            'state': state_name,
            'offender_type': 'Combined',
            'variable_category': row['variable_category'],
            'variable_detailed': row['variable_detailed'],
            'value': row['value'],
            'value_type': 'count',
            'value_source': 'calculated'
        })
    
    return pd.DataFrame(combined_rows)


def calculate_percentages(df_combined, state_name):
    """
    Calculate percentages for all demographic categories.
    Returns a dataframe of percentage rows to add.
    """
    # Get total profiles for each offender type
    totals_map = (df_combined[
        (df_combined['state'] == state_name) &
        (df_combined['variable_category'] == 'total') &
        (df_combined['variable_detailed'] == 'total_profiles')
    ].set_index('offender_type')['value'].to_dict())
    
    percentage_rows = []
    
    for offender_type, total in totals_map.items():
        # Get all demographic counts
        demo_data = df_combined[
            (df_combined['state'] == state_name) &
            (df_combined['offender_type'] == offender_type) &
            (df_combined['variable_category'].isin(['gender', 'race'])) &
            (df_combined['value_type'] == 'count')
        ]
        
        # Calculate percentage for each
        for _, row in demo_data.iterrows():
            percentage = (row['value'] / total) * 100
            percentage_rows.append({
                'state': state_name,
                'offender_type': offender_type,
                'variable_category': row['variable_category'],
                'variable_detailed': row['variable_detailed'],
                'value': round(percentage, 2),
                'value_type': 'percentage',
                'value_source': 'calculated'
            })
    
    return pd.DataFrame(percentage_rows)


def calculate_counts_from_percentages(df_combined, state_name):
    """
    Calculate counts from percentages when only percentages are provided.
    Returns a dataframe of count rows to add.
    """
    # Get total profiles for each offender type
    totals_map = (df_combined[
        (df_combined['state'] == state_name) &
        (df_combined['variable_category'] == 'total') &
        (df_combined['variable_detailed'] == 'total_profiles')
    ].set_index('offender_type')['value'].to_dict())
    
    count_rows = []
    
    for offender_type, total in totals_map.items():
        # Get all demographic percentages
        demo_data = df_combined[
            (df_combined['state'] == state_name) &
            (df_combined['offender_type'] == offender_type) &
            (df_combined['variable_category'].isin(['gender', 'race'])) &
            (df_combined['value_type'] == 'percentage')
        ]
        
        # Calculate count for each
        for _, row in demo_data.iterrows():
            count = round(total * (row['value'] / 100))
            count_rows.append({
                'state': state_name,
                'offender_type': offender_type,
                'variable_category': row['variable_category'],
                'variable_detailed': row['variable_detailed'],
                'value': int(count),
                'value_type': 'count',
                'value_source': 'calculated'
            })
    
    return pd.DataFrame(count_rows)


def standardize_offender_types(df):
    """
    Standardize offender type terminology across states.
    """
    replacements = {
        'Offenders': 'Convicted Offender',
        'Convicted offenders': 'Convicted Offender',
        'Arrested offender': 'Arrestee',
        'All': 'Combined'
    }
    
    df = df.copy()
    df['offender_type'] = df['offender_type'].replace(replacements)
    return df


def prepare_state_for_combined(df, state_name):
    """
    Prepare state data for appending to foia_combined.
    Selects only needed columns and adds value_source if missing.
    """
    df_prepared = df[COLUMNS_NEEDED].copy()
    
    # Add value_source if not present
    if 'value_source' not in df_prepared.columns:
        df_prepared['value_source'] = 'reported'
    else:
        df_prepared['value_source'] = df_prepared['value_source'].fillna('reported')
    
    return df_prepared


def format_compact(x, p=None):
    """
    Format numbers in compact notation (k for thousands, M for millions).
    
    Args:
        x: The number to format
        p: Position parameter (not used, but kept for compatibility with matplotlib)
    
    Returns:
        Formatted string with k/M suffix
    """
    if x >= 1000000:
        # For millions, show one decimal place if not a whole million
        return f'{x/1000000:.1f}M' if x/1000000 != int(x/1000000) else f'{int(x/1000000)}M'
    elif x >= 1000:
        # For thousands, always show as integer
        return f'{int(x/1000)}k'
    else:
        # For values under 1000, show as integer
        return f'{int(x)}'


def create_state_visualizations(df_combined, state_name):
    """
    Create count and percentage pie charts for a state.
    """
    state_data = df_combined[df_combined['state'] == state_name]
    
    # Determine offender types for this state
    offender_types = sorted(state_data['offender_type'].unique())
    n_types = len(offender_types)
    
    # Create figure
    fig, axes = plt.subplots(4, n_types, figsize=(5*n_types, 16))
    if n_types == 1:
        axes = axes.reshape(-1, 1)
    
    fig.suptitle(f'{state_name} DNA Database Demographics - Counts and Percentages', fontsize=18)
    
    # Create pie charts for each metric
    for i, offender_type in enumerate(offender_types):
        # Gender counts
        create_pie_chart(axes[0, i], state_data, offender_type, 'gender', 'count', 
                        f'{offender_type}\nGender Counts', show_values=True)
        
        # Gender percentages
        create_pie_chart(axes[1, i], state_data, offender_type, 'gender', 'percentage',
                        f'{offender_type}\nGender Percentages')
        
        # Race counts
        create_pie_chart(axes[2, i], state_data, offender_type, 'race', 'count',
                        f'{offender_type}\nRace Counts', show_values=True)
        
        # Race percentages
        create_pie_chart(axes[3, i], state_data, offender_type, 'race', 'percentage',
                        f'{offender_type}\nRace Percentages')
    
    plt.tight_layout()
    plt.show()
    
    # Display count summary
    print(f"\n{state_name} profile counts by offender type:")
    count_summary = state_data[
        (state_data['variable_category'] == 'total') &
        (state_data['variable_detailed'] == 'total_profiles') &
        (state_data['value_type'] == 'count')
    ][['offender_type', 'value']].set_index('offender_type')
    count_summary.columns = ['Total Profiles']
    count_summary['Total Profiles'] = count_summary['Total Profiles'].apply(lambda x: f"{x:,}")
    display(count_summary)


def create_pie_chart(ax, data, offender_type, category, value_type, title, show_values=False):
    """
    Helper to create a single pie chart.
    """
    chart_data = data[
        (data['offender_type'] == offender_type) &
        (data['variable_category'] == category) &
        (data['value_type'] == value_type)
    ].sort_values('value', ascending=False)
    
    if len(chart_data) > 0:
        if show_values and value_type == 'count':
            labels = [f"{row['variable_detailed']}\n({row['value']:,.0f})" 
                     for _, row in chart_data.iterrows()]
        else:
            labels = chart_data['variable_detailed']
        
        colors = plt.cm.Set3(range(len(chart_data)))
        wedges, texts, autotexts = ax.pie(chart_data['value'], 
                                          labels=labels,
                                          autopct='%1.1f%%',
                                          startangle=90,
                                          colors=colors)
        
        # Adjust text size for readability
        for text in texts:
            text.set_fontsize(10 if category == 'race' else 11)
        for autotext in autotexts:
            autotext.set_fontsize(10)
            
        ax.set_title(title)
    else:
        ax.text(0.5, 0.5, 'No data', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(title)


def verify_percentage_consistency(df_combined, state_name):
    """
    Verify that reported percentages match calculated percentages (within tolerance).
    Returns True if consistent, False otherwise.
    """
    state_data = df_combined[df_combined['state'] == state_name]
    
    # Get all offender types that have both counts and percentages
    offender_types = state_data['offender_type'].unique()
    
    consistency_results = []
    
    for offender_type in offender_types:
        offender_data = state_data[state_data['offender_type'] == offender_type]
        
        # Check if we have both reported and calculated percentages
        for category in ['gender', 'race']:
            reported_pcts = offender_data[
                (offender_data['variable_category'] == category) &
                (offender_data['value_type'] == 'percentage') &
                (offender_data['value_source'] == 'reported')
            ]
            
            calculated_pcts = offender_data[
                (offender_data['variable_category'] == category) &
                (offender_data['value_type'] == 'percentage') &
                (offender_data['value_source'] == 'calculated')
            ]
            
            if len(reported_pcts) > 0 and len(calculated_pcts) > 0:
                # Compare each demographic value
                for _, rep_row in reported_pcts.iterrows():
                    calc_match = calculated_pcts[
                        calculated_pcts['variable_detailed'] == rep_row['variable_detailed']
                    ]
                    if len(calc_match) > 0:
                        diff = abs(rep_row['value'] - calc_match.iloc[0]['value'])
                        consistency_results.append({
                            'offender_type': offender_type,
                            'category': category,
                            'variable': rep_row['variable_detailed'],
                            'reported': rep_row['value'],
                            'calculated': calc_match.iloc[0]['value'],
                            'difference': diff,
                            'consistent': diff < 0.5  # 0.5% tolerance
                        })
    
    if consistency_results:
        consistency_df = pd.DataFrame(consistency_results)
        print(f"\nPercentage consistency check for {state_name}:")
        print(f"All values consistent: {consistency_df['consistent'].all()}")
        
        if not consistency_df['consistent'].all():
            print("\nInconsistent values:")
            display(consistency_df[~consistency_df['consistent']])
        
        return consistency_df['consistent'].all()
    else:
        # No comparison possible - state only has one type of data
        return True


def create_demographic_bar_charts(df_combined, state_name):
    """
    Create clean bar charts showing demographics by offender type.
    Uses counts for exact values and includes percentage labels.
    """
    state_data = df_combined[df_combined['state'] == state_name]
    
    # Get offender types and ensure Combined is last
    offender_types = sorted(state_data[state_data['value_type'] == 'count']['offender_type'].unique())
    if 'Combined' in offender_types:
        offender_types.remove('Combined')
        offender_types.append('Combined')
    
    # Create figure with side-by-side subplots for gender and race
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 10))
    fig.suptitle(f'{state_name} DNA Database Demographics', fontsize=20, fontweight='bold', y=1.01)
    
    # Color palettes - using a modern, visually appealing palette
    gender_colors = {'Male': '#4E79A7', 'Female': '#E15759'}
    race_colors = {
        'White': '#4E79A7',
        'African American': '#F28E2B', 
        'Hispanic': '#E15759',
        'Asian': '#76B7B2',
        'Native American': '#59A14F',
        'Unknown': '#AF7AA1',
        'Other': '#9C755F'
    }
    
    # Gender subplot - horizontal bars
    ax1.set_title('Gender Distribution', fontsize=18, pad=15)
    gender_data = state_data[
        (state_data['variable_category'] == 'gender') &
        (state_data['value_type'] == 'count')
    ]
    
    # Prepare data for grouped horizontal bars
    y_positions = np.arange(len(offender_types))
    bar_height = 0.35
    
    # Process each gender
    for i, gender in enumerate(['Female', 'Male']):  # Female first to be on top
        values = []
        percentages = []
        
        for offender_type in offender_types:
            # Get count
            count_val = gender_data[
                (gender_data['offender_type'] == offender_type) &
                (gender_data['variable_detailed'] == gender)
            ]['value'].values
            
            # Get percentage
            pct_val = state_data[
                (state_data['offender_type'] == offender_type) &
                (state_data['variable_detailed'] == gender) &
                (state_data['variable_category'] == 'gender') &
                (state_data['value_type'] == 'percentage')
            ]['value'].values
            
            values.append(count_val[0] if len(count_val) > 0 else 0)
            percentages.append(pct_val[0] if len(pct_val) > 0 else 0)
        
        # Plot horizontal bars
        positions = y_positions + bar_height * (i - 0.5)
        bars = ax1.barh(positions, values, bar_height, label=gender, 
                        color=gender_colors.get(gender, '#333333'))
        
        # Add count and percentage labels to the right of bars
        for bar, pct, val in zip(bars, percentages, values):
            width = bar.get_width()
            if width > 0:
                # Put percentage and count to the right of the bar
                ax1.text(width, bar.get_y() + bar.get_height()/2,
                        f'  {pct:.1f}% ({int(val):,})', ha='left', va='center', fontsize=14)
    
    ax1.set_xlabel('Number of Profiles', fontsize=16)
    ax1.set_yticks(y_positions)
    ax1.set_yticklabels(offender_types, fontsize=14)
    ax1.legend(loc='upper right', fontsize=12)
    # Remove grid lines and spines for cleaner look
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    # Add some padding to x-axis for labels
    ax1.set_xlim(0, max([v for sublist in [gender_data[
        (gender_data['offender_type'] == ot) &
        (gender_data['variable_detailed'].isin(['Male', 'Female']))
    ]['value'].values for ot in offender_types] for v in sublist]) * 1.25)
    
    # Format x-axis with compact notation (k for thousands, M for millions)
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(format_compact))
    ax1.tick_params(axis='x', labelsize=12)
    ax1.invert_yaxis()  # Invert y-axis to have first offender type at top
    
    # Race subplot - horizontal bars
    ax2.set_title('Race Distribution', fontsize=18, pad=15)
    race_data = state_data[
        (state_data['variable_category'] == 'race') &
        (state_data['value_type'] == 'count')
    ]
    
    # Get all unique races for this state
    races = sorted(race_data['variable_detailed'].unique())
    
    # Adjust bar height based on number of races
    race_height = min(0.8 / len(races), 0.15)  # Cap at 0.15 to prevent too thick bars
    
    # Create race positions for grouping
    for i, race in enumerate(races):
        values = []
        percentages = []
        
        for offender_type in offender_types:
            # Get count
            count_val = race_data[
                (race_data['offender_type'] == offender_type) &
                (race_data['variable_detailed'] == race)
            ]['value'].values
            
            # Get percentage
            pct_val = state_data[
                (state_data['offender_type'] == offender_type) &
                (state_data['variable_detailed'] == race) &
                (state_data['variable_category'] == 'race') &
                (state_data['value_type'] == 'percentage')
            ]['value'].values
            
            values.append(count_val[0] if len(count_val) > 0 else 0)
            percentages.append(pct_val[0] if len(pct_val) > 0 else 0)
        
        # Plot horizontal bars
        positions = y_positions + race_height * (i - len(races)/2 + 0.5)
        bars = ax2.barh(positions, values, race_height, label=race,
                        color=race_colors.get(race, '#333333'))
        
        # Add count and percentage labels to the right of bars
        for bar, pct, val in zip(bars, percentages, values):
            width = bar.get_width()
            if width > 0:
                # Put percentage and count to the right of the bar
                ax2.text(width, bar.get_y() + bar.get_height()/2,
                        f'  {pct:.1f}% ({int(val):,})', ha='left', va='center', fontsize=12)
    
    ax2.set_xlabel('Number of Profiles', fontsize=16)
    ax2.set_yticks(y_positions)
    ax2.set_yticklabels(offender_types, fontsize=14)
    ax2.legend(loc='upper right', ncol=1, fontsize=11)
    # Remove grid lines and spines for cleaner look
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    # Add some padding to x-axis for labels
    ax2.set_xlim(0, max([v for sublist in [race_data[
        race_data['offender_type'] == ot
    ]['value'].values for ot in offender_types] for v in sublist if v > 0]) * 1.25)
    
    # Format x-axis with compact notation (k for thousands, M for millions)
    ax2.xaxis.set_major_formatter(plt.FuncFormatter(format_compact))
    ax2.tick_params(axis='x', labelsize=12)
    ax2.invert_yaxis()  # Invert y-axis to have first offender type at top
    
    plt.tight_layout()
    plt.show()
    
    # Also display a summary table with totals
    print(f"\n{state_name} profile totals by offender type:")
    totals_data = state_data[
        (state_data['variable_category'] == 'total') &
        (state_data['variable_detailed'] == 'total_profiles') &
        (state_data['value_type'] == 'count')
    ][['offender_type', 'value']].copy()
    totals_data['value'] = totals_data['value'].apply(lambda x: f"{int(x):,}")
    totals_data.columns = ['Offender Type', 'Total Profiles']
    display(totals_data.set_index('Offender Type'))

# Analyses

## California


In [ ]:
#| label: ca-load-preview
#| echo: true

ca_path = per_state / "california_foia_data.csv"
ca      = pd.read_csv(ca_path)

display(ca)

We begin by loading California's file. California supplies **counts only** for gender and race plus a separate total for each offender type; no percentages are reported.

Recording California's reporting structure:


In [ ]:
#| label: ca-add-metadata
#| echo: true

foia_state_metadata.append({
    "state": "California",
    "race_report_values":   report_status(ca, "race"),
    "gender_report_values": report_status(ca, "gender")
})

print("✓ Added metadata row:", foia_state_metadata[-1])

### Verifying that demographic counts match reported totals

Using the `verify_category_totals` helper function:


In [ ]:
#| label: ca-run-category-verify
#| echo: true

ca_category_qc = verify_category_totals(ca)
ca_category_qc

### Interpreting the race shortfall

> *"Racial classification is not considered a required field on the collection card; thus, an unknown number of offenders may have **no racial classification listed**."*
> — California DOJ FOIA letter, July 10 2018 (`raw/foia_pdfs/FOIA_RacialComp_California.pdf`)

The 393  887 Convicted‑Offender profiles and 96  127 Arrestee profiles that do **not** appear in any of the four reported race categories must therefore belong to an unreported "Unknown/Other" bucket. To keep row totals consistent and enable percentage calculations later, we add one new race row per offender type:

* `variable_category = "race"`
* `variable_detailed = "Unknown"`
* `value_type = "count"`
* `value_source = "calculated"`


In [ ]:
#| label: ca-insert-unknown
#| echo: true

# ------------------------------------------------------------------
# 1.  Remove any prior Unknown‑race rows 
# ------------------------------------------------------------------
ca = ca[~(
    (ca["variable_category"] == "race") &
    (ca["variable_detailed"] == "Unknown")
)].copy()

# ------------------------------------------------------------------
# 2.  Add one Unknown‑race row per offender type
# ------------------------------------------------------------------
unknown_rows = pd.DataFrame([
    {
        "state": "California",
        "offender_type": "Convicted Offender",
        "variable_category": "race",
        "variable_detailed": "Unknown",
        "value": 393887,           
        "value_type": "count",
        "value_source": "calculated",
    },
    {
        "state": "California",
        "offender_type": "Arrestee",
        "variable_category": "race",
        "variable_detailed": "Unknown",
        "value": 96127,
        "value_type": "count",
        "value_source": "calculated",
    },
])

ca = pd.concat([ca, unknown_rows], ignore_index=True)

# ------------------------------------------------------------------
# 3.  Confirm race and gender now reconcile to totals
# ------------------------------------------------------------------
print("Category‑level check after adding Unknown rows:")
display(verify_category_totals(ca))

### Creating Combined totals

California reported data separately for Convicted Offenders and Arrestees. Combined totals are calculated by summing across both offender types for all categories and demographic values.


In [ ]:
#| label: ca-create-combined
#| echo: true

# Calculate Combined totals using helper function
combined_df = calculate_combined_totals(ca, "California")

# Add Combined rows to ca
ca = pd.concat([ca, combined_df], ignore_index=True)

print("✓ Created Combined totals for California")
print(f"  Combined total profiles: {combined_df[combined_df['variable_detailed'] == 'total_profiles']['value'].values[0]:,}")

# Display Combined gender counts
print("\nCombined gender counts:")
display(combined_df[combined_df['variable_category'] == 'gender'].sort_values('variable_detailed'))

# Display Combined race counts  
print("\nCombined race counts:")
display(combined_df[combined_df['variable_category'] == 'race'].sort_values('variable_detailed'))

# Verify Combined calculations
print("\n✓ Verification of Combined calculations:")

# Get counts dataframe for verification
counts_df = ca[(ca['value_type'] == 'count') & (ca['offender_type'] != 'Combined')].copy()

conv_total = counts_df[(counts_df['offender_type'] == 'Convicted Offender') & 
                       (counts_df['variable_detailed'] == 'total_profiles')]['value'].values[0]
arr_total = counts_df[(counts_df['offender_type'] == 'Arrestee') & 
                      (counts_df['variable_detailed'] == 'total_profiles')]['value'].values[0]
comb_total = combined_df[combined_df['variable_detailed'] == 'total_profiles']['value'].values[0]

print(f"  Convicted Offender total: {conv_total:,}")
print(f"  Arrestee total: {arr_total:,}")
print(f"  Sum: {conv_total + arr_total:,}")
print(f"  Combined total: {comb_total:,}")
print(f"  Match: {conv_total + arr_total == comb_total}")

### Preparing California data for the combined dataset

The California data requires selection of specific columns for inclusion in the combined dataset.


In [ ]:
#| label: ca-append-to-combined
#| echo: true

# ------------------------------------------------------------------
# Create California data for foia_combined with only needed columns
# ------------------------------------------------------------------
# Select existing California data with the columns we want
ca_for_combined = ca[['state', 'offender_type', 'variable_category', 
                      'variable_detailed', 'value', 'value_type']].copy()

# Add value_source column
ca_for_combined['value_source'] = 'reported'

# Update value_source for calculated rows (Unknown race and Combined totals)
ca_for_combined.loc[
    ((ca_for_combined['variable_category'] == 'race') & 
     (ca_for_combined['variable_detailed'] == 'Unknown')) |
    (ca_for_combined['offender_type'] == 'Combined'), 
    'value_source'
] = 'calculated'

# Display sample of data structure
print("California data structure verification:")
print(f"Unique offender types: {ca_for_combined['offender_type'].unique()}")
print(f"Unique variable categories: {ca_for_combined['variable_category'].unique()}")

# Append to the combined dataframe
foia_combined = pd.concat([foia_combined, ca_for_combined], ignore_index=True)

print(f"\n✓ Appended {len(ca_for_combined)} California rows to foia_combined")
print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

### Calculating percentages

California provided only counts. For comparative analysis, percentages are calculated for each demographic category as the proportion of each demographic value relative to the total profiles for that offender type.


In [ ]:
#| label: ca-calculate-percentages
#| echo: true

# Calculate percentages using helper function
percentages_df = calculate_percentages(foia_combined, "California")

# Add percentages to foia_combined
foia_combined = pd.concat([foia_combined, percentages_df], ignore_index=True)

print(f"✓ Calculated and added {len(percentages_df)} percentage rows for California")
print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

# Display the actual percentage values
print("\nGender percentages by offender type:")
gender_pct = percentages_df[percentages_df['variable_category'] == 'gender'].pivot_table(
    index='variable_detailed', columns='offender_type', values='value'
)
display(gender_pct)

print("\nRace percentages by offender type:")
race_pct = percentages_df[percentages_df['variable_category'] == 'race'].pivot_table(
    index='variable_detailed', columns='offender_type', values='value'
)
display(race_pct)

# Verify percentages sum to approximately 100% for each category
print("\nVerification of percentage totals by category:")
verification = (percentages_df.groupby(['offender_type', 'variable_category'])['value']
                .sum()
                .round(2)
                .reset_index()
                .rename(columns={'value': 'sum_of_percentages'}))
display(verification)

# Note about rounding
print("\nNote: Percentage sums may not equal exactly 100% due to rounding to 2 decimal places.")

### Visualizing California demographics


In [ ]:
#| label: ca-visualize
#| echo: true

# Create cleaner bar chart visualization
create_demographic_bar_charts(foia_combined, "California")

### Summary of California processing

California data processing complete. The dataset now includes:

- Reported counts for Convicted Offenders and Arrestees
- Calculated Unknown race counts to reconcile reported totals
- Calculated Combined totals for gender and race categories
- Calculated percentages for all offender types and demographic categories

All values include appropriate `value_source` indicators distinguishing reported from calculated values.

## Florida


In [ ]:
#| label: fl-load-preview
#| echo: true

fl_path = per_state / "florida_foia_data.csv"
fl = pd.read_csv(fl_path)

display(fl)

Florida provides both counts and percentages for gender and race categories. The state already includes Combined totals, simplifying our processing.

Recording Florida's reporting structure in our metadata:


In [ ]:
#| label: fl-add-metadata
#| echo: true

foia_state_metadata.append({
    "state": "Florida",
    "race_report_values": report_status(fl, "race"),
    "gender_report_values": report_status(fl, "gender")
})

print("✓ Added metadata row:", foia_state_metadata[-1])

### Verifying demographic consistency


In [ ]:
#| label: fl-verify-totals
#| echo: true

fl_category_qc = verify_category_totals(fl)
display(fl_category_qc)

Florida's demographic counts sum correctly to the reported total. The state data appears internally consistent.

### Preparing Florida data for the combined dataset


In [ ]:
#| label: fl-append-to-combined
#| echo: true

# Prepare Florida data with only needed columns
fl_for_combined = prepare_state_for_combined(fl, "Florida")

# Display structure verification
print("Florida data structure verification:")
print(f"Unique offender types: {fl_for_combined['offender_type'].unique()}")
print(f"Unique variable categories: {fl_for_combined['variable_category'].unique()}")
print(f"Value types present: {fl_for_combined['value_type'].unique()}")

# Append to the combined dataframe
foia_combined = pd.concat([foia_combined, fl_for_combined], ignore_index=True)

print(f"\n✓ Appended {len(fl_for_combined)} Florida rows to foia_combined")
print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

### Visualizing Florida demographics


In [ ]:
#| label: fl-visualize
#| echo: true

create_demographic_bar_charts(foia_combined, "Florida")

### Summary of Florida processing

Florida data processing complete. The state provided:
- Both counts and percentages for all demographic categories
- Combined totals already calculated
- Internally consistent data requiring no adjustments

All values maintain `value_source = "reported"` as no calculations were necessary.

## Indiana


In [ ]:
#| label: in-load-preview
#| echo: true

in_path = per_state / "indiana_foia_data.csv"
indiana = pd.read_csv(in_path)

display(indiana)

Indiana presents a unique reporting pattern: total counts are provided, but demographic breakdowns are given only as percentages. Additionally, the value column contains String data that requires conversion.


In [ ]:
#| label: in-convert-values
#| echo: true

# First, let's examine the data to see what values we have
print("Unique values in Indiana data before conversion:")
print(indiana['value'].unique())

# Convert string values to numeric, handling "<1" as 0.5
indiana['value'] = indiana['value'].apply(lambda x: 0.5 if x == '<1' else pd.to_numeric(x))

print("\n✓ Converted Indiana values from String to numeric")
print(f"Value data type after conversion: {indiana['value'].dtype}")
print(f"Values after conversion: {indiana['value'].unique()}")

Recording Indiana's reporting structure:


In [ ]:
#| label: in-add-metadata
#| echo: true

foia_state_metadata.append({
    "state": "Indiana",
    "race_report_values": report_status(indiana, "race"),
    "gender_report_values": report_status(indiana, "gender"),
    "notes": "Provides percentages for demographics, counts for totals only"
})

print("✓ Added metadata row:", foia_state_metadata[-1])

### Understanding Indiana's unique reporting structure

Indiana provides an unusual reporting pattern:
- Total profile counts for Convicted Offender and Arrestee separately
- Demographic percentages only for Combined totals (not broken down by offender type)

First, let's calculate the Combined total:


In [ ]:
#| label: in-calculate-combined-total
#| echo: true

# Calculate Combined total from the separate offender type totals
conv_total = indiana[(indiana['offender_type'] == 'Convicted Offender') & 
                     (indiana['variable_detailed'] == 'total_profiles')]['value'].values[0]
arr_total = indiana[(indiana['offender_type'] == 'Arrestee') & 
                    (indiana['variable_detailed'] == 'total_profiles')]['value'].values[0]
combined_total = conv_total + arr_total

print(f"Convicted Offender total: {conv_total:,.0f}")
print(f"Arrestee total: {arr_total:,.0f}")
print(f"Combined total: {combined_total:,.0f}")

# Add Combined total to Indiana data
combined_total_row = pd.DataFrame([{
    'state': 'Indiana',
    'offender_type': 'Combined',
    'variable_category': 'total',
    'variable_detailed': 'total_profiles',
    'value': combined_total,
    'value_type': 'count',
    'value_source': 'calculated'
}])

indiana = pd.concat([indiana, combined_total_row], ignore_index=True)
print("\n✓ Added Combined total to Indiana data")

### Calculating demographic counts from percentages

Now we can calculate the actual counts from the Combined percentages:


In [ ]:
#| label: in-prepare-and-calculate
#| echo: true

# First, prepare and append the reported data
in_for_combined = prepare_state_for_combined(indiana, "Indiana")

# Update value_source for the calculated Combined total
in_for_combined.loc[
    (in_for_combined['offender_type'] == 'Combined') & 
    (in_for_combined['variable_detailed'] == 'total_profiles'),
    'value_source'
] = 'calculated'

foia_combined = pd.concat([foia_combined, in_for_combined], ignore_index=True)

print(f"✓ Appended {len(in_for_combined)} Indiana reported rows to foia_combined")

# Now calculate counts from the percentages
in_counts = calculate_counts_from_percentages(foia_combined, "Indiana")

# Append calculated counts
if len(in_counts) > 0:
    foia_combined = pd.concat([foia_combined, in_counts], ignore_index=True)
    print(f"✓ Calculated and added {len(in_counts)} count rows for Indiana")
    
    # Display sample of calculated counts
    print("\nSample of calculated Indiana counts:")
    display(in_counts[in_counts['variable_category'] == 'race'].head())
else:
    print("✗ No counts could be calculated (this shouldn't happen)")

print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

### Verifying calculated totals


In [ ]:
#| label: in-verify-calculated
#| echo: true

# Create temporary dataframe with Indiana's complete data for verification
in_complete = pd.concat([indiana, in_counts], ignore_index=True)
in_category_qc = verify_category_totals(in_complete)
display(in_category_qc)

# Check if percentages sum to ~100%
print("\nVerifying Indiana percentages sum to ~100%:")
in_pct_check = (indiana[indiana['value_type'] == 'percentage']
                .groupby(['offender_type', 'variable_category'])['value']
                .sum()
                .round(2))
display(in_pct_check)

### Visualizing Indiana demographics


In [ ]:
#| label: in-visualize
#| echo: true

create_demographic_bar_charts(foia_combined, "Indiana")

### Summary of Indiana processing

Indiana data processing complete. The state's unique reporting required:
- Conversion of String values to numeric format
- Calculation of demographic counts from reported percentages
- All demographic values now available in both count and percentage formats

Mixed `value_source` attribution: percentages are "reported", counts are "calculated".

## Maine


In [ ]:
#| label: me-load-preview
#| echo: true

me_path = per_state / "maine_foia_data.csv"
maine = pd.read_csv(me_path)

display(maine)

Maine provides comprehensive reporting with both counts and percentages for all demographic categories, including Combined totals.

Recording Maine's reporting structure:


In [ ]:
#| label: me-add-metadata
#| echo: true

foia_state_metadata.append({
    "state": "Maine",
    "race_report_values": report_status(maine, "race"),
    "gender_report_values": report_status(maine, "gender")
})

print("✓ Added metadata row:", foia_state_metadata[-1])

### Verifying demographic consistency


In [ ]:
#| label: me-verify-totals
#| echo: true

me_category_qc = verify_category_totals(maine)
display(me_category_qc)

# Also verify that reported percentages sum to ~100%
print("\nVerifying Maine percentages sum to ~100%:")
me_pct_check = (maine[maine['value_type'] == 'percentage']
                .groupby(['variable_category'])['value']
                .sum()
                .round(2))
display(me_pct_check)

Maine's data shows excellent internal consistency with both counts and percentages properly aligned.

### Preparing Maine data for the combined dataset


In [ ]:
#| label: me-append-to-combined
#| echo: true

# Prepare Maine data with only needed columns
me_for_combined = prepare_state_for_combined(maine, "Maine")

# Display structure verification
print("Maine data structure verification:")
print(f"Unique offender types: {me_for_combined['offender_type'].unique()}")
print(f"Unique variable categories: {me_for_combined['variable_category'].unique()}")
print(f"Value types present: {me_for_combined['value_type'].unique()}")

# Append to the combined dataframe
foia_combined = pd.concat([foia_combined, me_for_combined], ignore_index=True)

print(f"\n✓ Appended {len(me_for_combined)} Maine rows to foia_combined")
print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

### Visualizing Maine demographics


In [ ]:
#| label: me-visualize
#| echo: true

create_demographic_bar_charts(foia_combined, "Maine")

### Summary of Maine processing

Maine data processing complete. The state provided:
- Complete reporting with both counts and percentages
- Combined totals already calculated
- Internally consistent data with percentages summing to 100%

All values maintain `value_source = "reported"` as no calculations were necessary.

## Nevada


In [ ]:
#| label: nv-load-preview
#| echo: true

nv_path = per_state / "nevada_foia_data.csv"
nevada = pd.read_csv(nv_path)

display(nevada)

Nevada uses non-standard terminology that requires standardization:
- "All" instead of "Combined"
- "Arrested offender" instead of "Arrestee"
- "Convicted offenders" instead of "Convicted Offender"
- Uses "flags" terminology instead of "profiles"


In [ ]:
#| label: nv-standardize
#| echo: true

# Standardize offender types
nevada = standardize_offender_types(nevada)

print("✓ Standardized Nevada offender type terminology")
print(f"Offender types after standardization: {sorted(nevada['offender_type'].unique())}")

Recording Nevada's reporting structure:


In [ ]:
#| label: nv-add-metadata
#| echo: true

foia_state_metadata.append({
    "state": "Nevada",
    "race_report_values": report_status(nevada, "race"),
    "gender_report_values": report_status(nevada, "gender"),
    "notes": "Uses 'flags' terminology instead of 'profiles'"
})

print("✓ Added metadata row:", foia_state_metadata[-1])

### Verifying demographic consistency


In [ ]:
#| label: nv-verify-totals
#| echo: true

nv_category_qc = verify_category_totals(nevada)
display(nv_category_qc)

# Verify percentages sum to ~100%
print("\nVerifying Nevada percentages sum to ~100%:")
nv_pct_check = (nevada[nevada['value_type'] == 'percentage']
                .groupby(['offender_type', 'variable_category'])['value']
                .sum()
                .round(2))
display(nv_pct_check)

Nevada's data shows good internal consistency after terminology standardization.

### Preparing Nevada data for the combined dataset


In [ ]:
#| label: nv-append-to-combined
#| echo: true

# Prepare Nevada data with only needed columns
nv_for_combined = prepare_state_for_combined(nevada, "Nevada")

# Display structure verification
print("Nevada data structure verification:")
print(f"Unique offender types: {nv_for_combined['offender_type'].unique()}")
print(f"Unique variable categories: {nv_for_combined['variable_category'].unique()}")
print(f"Value types present: {nv_for_combined['value_type'].unique()}")

# Append to the combined dataframe
foia_combined = pd.concat([foia_combined, nv_for_combined], ignore_index=True)

print(f"\n✓ Appended {len(nv_for_combined)} Nevada rows to foia_combined")
print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

### Visualizing Nevada demographics


In [ ]:
#| label: nv-visualize
#| echo: true

create_demographic_bar_charts(foia_combined, "Nevada")

### Summary of Nevada processing

Nevada data processing complete. The state required:
- Standardization of offender type terminology for consistency
- Recognition of "flags" as equivalent to "profiles" in other states
- Both counts and percentages were provided for all categories

All values maintain `value_source = "reported"` as only terminology changes were necessary.

## South Dakota


In [ ]:
#| label: sd-load-preview
#| echo: true

sd_path = per_state / "south_dakota_foia_data.csv"
south_dakota = pd.read_csv(sd_path)

display(south_dakota.head(20))
print(f"\n... showing first 20 of {len(south_dakota)} rows")

South Dakota provides the most comprehensive reporting with 41 rows, including:
- Standard gender and race breakdowns
- Unique `gender_race` cross-tabulation showing intersectional demographics
- Both counts and percentages for all categories

Recording South Dakota's reporting structure:


In [ ]:
#| label: sd-add-metadata
#| echo: true

foia_state_metadata.append({
    "state": "South Dakota",
    "race_report_values": report_status(south_dakota, "race"),
    "gender_report_values": report_status(south_dakota, "gender"),
    "notes": "Includes gender_race cross-tabulation for intersectional analysis"
})

print("✓ Added metadata row:", foia_state_metadata[-1])

### Examining the intersectional data


In [ ]:
#| label: sd-examine-intersectional
#| echo: true

# Display the unique gender_race combinations
gender_race_data = south_dakota[south_dakota['variable_category'] == 'gender_race']
print("South Dakota's intersectional gender × race categories:")
print(f"Total gender_race rows: {len(gender_race_data)}")
print("\nUnique combinations:")
for combo in sorted(gender_race_data['variable_detailed'].unique()):
    print(f"  • {combo}")

### Verifying demographic consistency


In [ ]:
#| label: sd-verify-totals
#| echo: true

sd_category_qc = verify_category_totals(south_dakota)
display(sd_category_qc)

# Verify percentages sum to ~100%
print("\nVerifying South Dakota percentages sum to ~100%:")
sd_pct_check = (south_dakota[south_dakota['value_type'] == 'percentage']
                .groupby(['variable_category'])['value']
                .sum()
                .round(2))
display(sd_pct_check)

South Dakota's data demonstrates excellent internal consistency across all categories, including the intersectional data.

### Preparing South Dakota data for the combined dataset


In [ ]:
#| label: sd-append-to-combined
#| echo: true

# Prepare South Dakota data with only needed columns
sd_for_combined = prepare_state_for_combined(south_dakota, "South Dakota")

# Display structure verification
print("South Dakota data structure verification:")
print(f"Unique offender types: {sd_for_combined['offender_type'].unique()}")
print(f"Unique variable categories: {sd_for_combined['variable_category'].unique()}")
print(f"Value types present: {sd_for_combined['value_type'].unique()}")

# Show breakdown by category
print("\nRows by variable category:")
print(sd_for_combined['variable_category'].value_counts())

# Append to the combined dataframe
foia_combined = pd.concat([foia_combined, sd_for_combined], ignore_index=True)

print(f"\n✓ Appended {len(sd_for_combined)} South Dakota rows to foia_combined")
print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

### Visualizing South Dakota demographics


In [ ]:
#| label: sd-visualize
#| echo: true

create_demographic_bar_charts(foia_combined, "South Dakota")

### Summary of South Dakota processing

South Dakota data processing complete. The state provided:
- Most comprehensive reporting with 41 rows of data
- Standard gender and race categories plus unique intersectional gender×race data
- Complete counts and percentages for all categories
- Exemplary data quality with perfect internal consistency

All values maintain `value_source = "reported"`. The intersectional data provides unique insights unavailable from other states.

## Texas


In [ ]:
#| label: tx-load-preview
#| echo: true

tx_path = per_state / "texas_foia_data.csv"
texas = pd.read_csv(tx_path)

display(texas)

Texas provides counts only and uses "Offenders" instead of "Convicted Offender". Like California, Texas will need Combined totals and percentages calculated.


In [ ]:
#| label: tx-standardize
#| echo: true

# Standardize offender types
texas = standardize_offender_types(texas)

print("✓ Standardized Texas offender type terminology")
print(f"Offender types after standardization: {sorted(texas['offender_type'].unique())}")

Recording Texas's reporting structure:


In [ ]:
#| label: tx-add-metadata
#| echo: true

foia_state_metadata.append({
    "state": "Texas",
    "race_report_values": report_status(texas, "race"),
    "gender_report_values": report_status(texas, "gender")
})

print("✓ Added metadata row:", foia_state_metadata[-1])

### Verifying demographic consistency


In [ ]:
#| label: tx-verify-totals
#| echo: true

tx_category_qc = verify_category_totals(texas)
display(tx_category_qc)

Texas shows perfect internal consistency with all demographic counts matching reported totals.

### Creating Combined totals


In [ ]:
#| label: tx-create-combined
#| echo: true

# Calculate Combined totals using helper function
tx_combined = calculate_combined_totals(texas, "Texas")

# Add Combined rows to texas dataframe
texas = pd.concat([texas, tx_combined], ignore_index=True)

print("✓ Created Combined totals for Texas")
print(f"  Combined total profiles: {tx_combined[tx_combined['variable_detailed'] == 'total_profiles']['value'].values[0]:,}")

# Display Combined counts
print("\nCombined gender counts:")
display(tx_combined[tx_combined['variable_category'] == 'gender'].sort_values('variable_detailed'))

print("\nCombined race counts:")
display(tx_combined[tx_combined['variable_category'] == 'race'].sort_values('variable_detailed'))

# Verify Combined calculations
print("\n✓ Verification of Combined calculations:")

# Get original counts for verification
conv_total = texas[(texas['offender_type'] == 'Convicted Offender') & 
                   (texas['variable_detailed'] == 'total_profiles') &
                   (texas['value_type'] == 'count')]['value'].values[0]
arr_total = texas[(texas['offender_type'] == 'Arrestee') & 
                  (texas['variable_detailed'] == 'total_profiles') &
                  (texas['value_type'] == 'count')]['value'].values[0]
comb_total = tx_combined[tx_combined['variable_detailed'] == 'total_profiles']['value'].values[0]

print(f"  Convicted Offender total: {conv_total:,}")
print(f"  Arrestee total: {arr_total:,}")
print(f"  Sum: {conv_total + arr_total:,}")
print(f"  Combined total: {comb_total:,}")
print(f"  Match: {conv_total + arr_total == comb_total}")

### Preparing Texas data for the combined dataset


In [ ]:
#| label: tx-append-to-combined
#| echo: true

# Prepare Texas data with only needed columns
tx_for_combined = prepare_state_for_combined(texas, "Texas")

# Update value_source for Combined rows
tx_for_combined.loc[
    tx_for_combined['offender_type'] == 'Combined', 
    'value_source'
] = 'calculated'

# Display structure verification
print("Texas data structure verification:")
print(f"Unique offender types: {tx_for_combined['offender_type'].unique()}")
print(f"Unique variable categories: {tx_for_combined['variable_category'].unique()}")

# Append to the combined dataframe
foia_combined = pd.concat([foia_combined, tx_for_combined], ignore_index=True)

print(f"\n✓ Appended {len(tx_for_combined)} Texas rows to foia_combined")
print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

### Calculating percentages


In [ ]:
#| label: tx-calculate-percentages
#| echo: true

# Calculate percentages using helper function
tx_percentages = calculate_percentages(foia_combined, "Texas")

# Add percentages to foia_combined
foia_combined = pd.concat([foia_combined, tx_percentages], ignore_index=True)

print(f"✓ Calculated and added {len(tx_percentages)} percentage rows for Texas")
print(f"✓ Total rows in foia_combined: {len(foia_combined)}")

# Display sample of calculated percentages
print("\nGender percentages by offender type:")
gender_pct = tx_percentages[tx_percentages['variable_category'] == 'gender'].pivot_table(
    index='variable_detailed', columns='offender_type', values='value'
)
display(gender_pct)

# Verify percentages sum to ~100%
print("\nVerification of percentage totals by category:")
verification = (tx_percentages.groupby(['offender_type', 'variable_category'])['value']
                .sum()
                .round(2)
                .reset_index()
                .rename(columns={'value': 'sum_of_percentages'}))
display(verification)

### Visualizing Texas demographics


In [ ]:
#| label: tx-visualize
#| echo: true

create_state_visualizations(foia_combined, "Texas")

### Summary of Texas processing

Texas data processing complete. The state required:
- Standardization of "Offenders" to "Convicted Offender"
- Calculation of Combined totals across offender types
- Calculation of percentages from provided counts

Mixed `value_source` attribution: original counts are "reported", Combined totals and all percentages are "calculated".

# Conclusions

## Final Processing Summary

All seven states have been successfully processed and combined into a unified dataset.


In [ ]:
#| label: final-summary-all-states
#| echo: true

# Display final metadata summary
print("State Processing Metadata:")
print("=" * 80)
metadata_df = pd.DataFrame(foia_state_metadata)
display(metadata_df)

# Summary statistics
print(f"\n✓ Total rows in foia_combined: {len(foia_combined)}")
print(f"✓ States processed: {sorted(foia_combined['state'].unique())}")
print(f"✓ Offender types: {sorted(foia_combined['offender_type'].unique())}")
print(f"✓ Variable categories: {sorted(foia_combined['variable_category'].unique())}")

# Value source breakdown
print("\nValue source breakdown:")
source_counts = foia_combined['value_source'].value_counts()
for source, count in source_counts.items():
    pct = (count / len(foia_combined)) * 100
    print(f"  • {source}: {count} rows ({pct:.1f}%)")

# Show sample of final combined data
print("\nSample of final combined dataset:")
display(foia_combined.sample(10, random_state=42))

## Saving Processed Data


In [ ]:
#| label: save-outputs
#| echo: true

# Define output paths
output_dir = base_dir / "output" / "foia"
output_dir.mkdir(parents=True, exist_ok=True)

# Save the combined dataset
foia_output_path = output_dir / "foia_data_clean.csv"
foia_combined.to_csv(foia_output_path, index=False)
print(f"✓ Saved combined FOIA data to: {foia_output_path}")

# Save the metadata
metadata_output_path = output_dir / "foia_state_metadata.csv"
metadata_df.to_csv(metadata_output_path, index=False)
print(f"✓ Saved state metadata to: {metadata_output_path}")

# Create final frozen version
frozen_dir = base_dir / "data" / "v1.0"
frozen_dir.mkdir(parents=True, exist_ok=True)

frozen_path = frozen_dir / "foia_state_race_v1.0.csv"
foia_combined.to_csv(frozen_path, index=False)
print(f"✓ Created frozen version at: {frozen_path}")

print("\n✅ All processing complete!")

## Key Findings and Notes

1. **Data Completeness**: All states provided total profile counts and demographic breakdowns, though reporting formats varied significantly.

2. **Calculated Values**: 
   - California and Texas required Combined totals and percentage calculations
   - Indiana required count calculations from percentages
   - All calculated values are clearly marked with `value_source = "calculated"`

3. **Terminology Standardization**:
   - Nevada: "All" → "Combined", "Arrested offender" → "Arrestee"
   - Texas: "Offenders" → "Convicted Offender"

4. **Unique Features**:
   - South Dakota provided intersectional gender×race data unavailable from other states
   - Nevada uses "flags" terminology instead of "profiles"
   - California acknowledged missing race data, requiring "Unknown" category calculation

5. **Data Quality**: After processing, all states show internally consistent data with demographic counts summing to reported totals and percentages summing to approximately 100% (accounting for rounding).

This standardized dataset enables direct state-to-state comparisons of DNA database demographics while maintaining full transparency about data sources and calculations.

This processing pipeline has successfully transformed heterogeneous FOIA responses from seven states into a standardized, analysis-ready dataset. By maintaining clear attribution of reported versus calculated values and documenting all processing decisions, this work supports reproducible research on racial disparities in DNA databases.

The unified dataset (`foia_state_race_v1.0.csv`) and accompanying metadata provide researchers with accessible, transparent data to examine critical questions about equity and representation in forensic DNA databases. As noted in the introduction, making this data more accessible represents an important contribution to research ethics and transparency in criminal justice studies.